In [3]:
from zero.svd import MangakiSVD
from zero.als import MangakiALS
from zero.pca import MangakiPCA
from zero.knn import MangakiKNN
import numpy as np
#from mangaki.utils.values import rating_values
from zero.dataset import Dataset
#from mangaki.utils.fit_algo import fit_algo, get_algo_backup

# dataset = Dataset()
# dataset.load_csv('/Users/jilljenn/code/mangaki/mangaki/data/ratings.csv', convert=lambda choice: rating_values[choice], title_filename='works.csv')
# titles = dataset.titles

In [2]:
triplets = Rating.objects.values_list('user_id', 'work_id', 'choice')
dataset = Dataset()
anonymized = dataset.make_anonymous_data(triplets)

algo = MangakiALS(nb_iterations=10)
algo.set_parameters(dataset.anonymized.nb_users, dataset.anonymized.nb_works)
#algo.save('/tmp')
algo.load('/tmp')
#algo.fit(dataset.anonymized.X, dataset.anonymized.y)

print(algo.predict(dataset.anonymized.X)[:6])
print(dataset.anonymized.y[:6])

NameError: name 'Rating' is not defined

In [4]:
algo.U.shape

NameError: name 'algo' is not defined

In [ ]:
algo.VT.shape

## Find popular works and users

In [ ]:
from collections import Counter

NB = 50

popular_users = Counter([user_id for user_id, _ in dataset.anonymized.X])
popular_works = Counter([work_id for _, work_id in dataset.anonymized.X])

popular_user_ids = [user_id for user_id, _ in popular_users.most_common(NB)]
popular_work_ids = [work_id for work_id, _ in popular_works.most_common(NB)]

In [ ]:
import pandas as pd

df = pd.DataFrame(np.array(Work.objects.values_list('id', 'title')), columns=('work_id', 'title'))

In [ ]:
df.head()
df['work_id'] = df['work_id'].astype(np.int32)
df['encoded_work_id'] = df['work_id'].map(dataset.encode_work)

In [ ]:
df = df.dropna()

In [ ]:
df['encoded_work_id'] = df['encoded_work_id'].astype(np.int32)

In [ ]:
df.head()

In [ ]:
dataset.titles = dict(np.array(df[['encoded_work_id', 'title']]))

## Visualize anime embeddings using matplotlib

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD
plt.style.use('dark_background')

svd = TruncatedSVD()

class MangakiViz:
    def __init__(self, title):
        self.fig, self.ax = plt.subplots()
        plt.xlim((-1, 1))
        plt.ylim((-1, 1))
        plt.title(title)
        
    def load_points(self, V, displayed_ids, marker='.', color='#FF6A00', titles=None):
        V2 = svd.fit_transform(V)
        self.ax.scatter(V2[displayed_ids, 0], V2[displayed_ids, 1], marker=marker, c=color, picker=True)
        if titles is not None:
            self.V2 = V2
            self.displayed_ids = displayed_ids
            self.title_of_point = dict([(tuple(self.V2[entity_id].tolist()), titles[entity_id] if entity_id != 3 else 'Princess Mononoke') for entity_id in self.displayed_ids])
            self.fig.canvas.mpl_connect('pick_event', self.onpick)
    
    def display_title(self, x, y):
        plt.annotate(self.title_of_point[(x, y)], xy=(x, y), xycoords='data',
            xytext=(-30, -30), textcoords='offset points',
            arrowprops=dict(arrowstyle="->",
                            connectionstyle="arc3,rad=.2")
            )
    
    def onpick(self, event):
        ind = self.displayed_ids[event.ind[0]]
        x, y = self.V2[ind]
        self.display_title(x, y)
    
    def show_popular(self):
        for famous_pos in [0, 3, 5]:
            x, y = self.V2[self.displayed_ids[famous_pos]]
            self.display_title(x, y)
    
    def save(self, filename):
        print(filename)
        plt.show()
        # self.fig.savefig(filename, format='pdf')
        # plt.close()

U = algo.U
V = algo.VT.T
titles = dataset.titles
viz = MangakiViz('Visualizing anime embeddings')
viz.load_points(U, popular_user_ids)
viz.load_points(V, popular_work_ids, '+', 'tab:cyan', titles)
viz.show_popular()
viz.save('test.pdf')

## This below was just to generate the slides for Hacker News Kansai

In [ ]:
V_ids = np.arange(20).repeat(2)
U_ids = V_ids.copy()[1:]

In [ ]:
with open('/tmp/log.txt') as f:
    plot_titles = ['Start with random points'] + f.read().splitlines()

for t, (uid_t, vid_t) in enumerate(zip(U_ids, V_ids)):
    print(t, uid_t, vid_t, plot_titles[t])
    viz = MangakiViz(plot_titles[t])
    U = np.load('U{:d}.npy'.format(uid_t))
    V = np.load('V{:d}.npy'.format(vid_t))
    viz.load_points(U, popular_user_ids)
    viz.load_points(V, popular_work_ids, '+', 'tab:cyan', titles)
    viz.show_popular()
    viz.save('/Users/jilljenn/Desktop/hnkansai/figures/embed{:d}.pdf'.format(t))